In [ ]:
import json
import pandas as pd
import geopandas as gpd
import altair as alt

import vega_datasets
from ast import literal_eval

import numpy as np
import folium
from folium import Map, FeatureGroup, Marker, LayerControl
from folium.plugins import MeasureControl
import branca.colormap as cm

from localtileserver import get_folium_tile_layer
from localtileserver import TileClient

In [ ]:
def array_eval(x):
    x = x.replace('[', '')
    x = x.replace(']', '')
    x = np.fromstring(x, sep=' ')

    return x

In [ ]:
#gdf = gpd.read_file(r"C:\BERATools\Test-data\footprint-fixed-avg.geojson")
gdf = gpd.read_file(r"C:\BERATools\Surmont_Old_AOI\fixed_width\footprint_fixed_width.geojson")

In [ ]:
gdf['width_hist'] = gdf['width_hist'].apply(array_eval)
gdf['width_bins'] = gdf['width_bins'].apply(array_eval)

In [ ]:
def add_histgram(hist, bins, avg_width):
    histogram = pd.DataFrame({'hist': hist, 
                              'bins': bins, 
                              'avg_width':avg_width, 
                              'label': 'Width'})

    hist = alt.Chart(histogram).mark_bar().encode(
        x = 'bins',
        y = 'hist'
    )

    chart_two = alt.Chart(histogram).mark_rule().encode(
        x='avg_width',
        size=alt.value(8),
        color=alt.ColorValue('green')
    )

    label_text = chart_two.mark_text(
        align="left",
        baseline="middle",
        fontSize=14,
        dx=12
    ).encode(text="avg_width")
 
    vega_lite = folium.VegaLite(
        (hist+chart_two+label_text),
        width="100%",
        height="100%",
    )

    return vega_lite


Prepare raster layers

In [ ]:
# First, create a tile server from local raster file
tile_client = TileClient(r"C:\BERATools\Test-data\CHM.tif")
t = get_folium_tile_layer(tile_client)
t.layer_name = "CHM Layer"
t.control=True

# Study sites
# tile_4203 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4203_transparent_mosaic_Shifted.tif")
# t_4203 = get_folium_tile_layer(tile_4203)
# t_4203.control=True

# tile_4207 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4207_transparent_mosaic_Shifted.tif")
# t_4207 = get_folium_tile_layer(tile_4207)
# t_4207.control=True

# tile_4209 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4209_transparent_mosaic_Shifted.tif")
# t_4209 = get_folium_tile_layer(tile_4209)
# t_4209.control=True

In [ ]:
# Create two maps.
m = folium.Map(
    location=[56.201831,-110.672306],
    zoom_start=14,
    position="absolute",
    left="0%"
)

colormap = cm.linear.Set1_09.scale(min(gdf.avg_width), max(gdf.avg_width)).to_step(10)
colormap.caption = "A colormap caption"
m.add_child(colormap)

featuregrp_fixed_fp = FeatureGroup(name='Fixed widthe footprint')

for index, row in gdf.iterrows(): 
    polygon = folium.GeoJson(
        row['geometry'],
        style_function=lambda feature: {
            "fillColor": colormap(row['avg_width']),
            "color": "blue",
            "weight": 1,},
    )
          
    popup = folium.Popup()
    vega_lite = add_histgram(row['width_hist'], row['width_bins'], row['avg_width'])
    vega_lite.add_to(popup)
    popup.add_to(polygon)

    tooltip = folium.Tooltip(row['avg_width'])
    tooltip.add_to(polygon)

    featuregrp_fixed_fp.add_child(polygon)
    #polygon.add_to(m)

In [ ]:
m.add_child(t)
# m.add_child(t_4203)
# m.add_child(t_4207)
# t_4209.add_to(m)

featuregrp_fixed_fp.add_to(m)
m.add_child(MeasureControl())
folium.LayerControl().add_to(m)
m